In [1]:
import numpy as np
from scipy import sparse, stats
import pandas as pd
import bottleneck
import gc
import warnings
import scanpy as sc
import anndata as ad
import time
import os
import concurrent.futures
import threading
import concurrent.futures
from scipy.sparse import csr_matrix

from pyprojroot import here
import pymn
import resource

/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/ho

In [2]:
here("data")

PosixPath('/vault/lfrench/mouse_brain_cluster_replicability/data')

In [3]:
here()
base_data_folder = here("data")
#was also setup for metaclusters/subclass
level = "cluster"

In [4]:
#need full run locations for second best hits from 1v1 AUROCs
if level == "cluster":
    full_run_location = here("results/full_run_ZengAWS.1698256033/")
    metamarkers_location = "2009_reciprocals"


In [5]:
def get_auroc(scores, labels):
    ranks = bottleneck.rankdata(scores).astype(scores.dtype)
    n_pos = sum(labels) #could be precalculated for speed
    n_neg = len(ranks) - n_pos
    sum_pos_ranks = labels @ ranks
    auroc = sum_pos_ranks / n_pos
    auroc -= (n_pos + 1) / 2
    auroc /= n_neg
    return auroc
def getStudyId(text):
    return text.split("|")[0]

In [6]:
base_data_folder

PosixPath('/vault/lfrench/mouse_brain_cluster_replicability/data')

In [7]:
metamarkers_location

'2009_reciprocals'

In [8]:
#load metamarkers
meta_markers = pd.read_csv(here("results/MetaMarkers_2009_reciprocals/MetaMarkers_default_recurrence_2.csv.gz"))

meta_markers["Zeng_cluster"] = meta_markers.cell_type.str.split("__", expand=True)[[0]]
meta_markers["Macosko_cluster"] = meta_markers.cell_type.str.split("__", expand=True)[[1]]

#should be sorted already but this ensures it
meta_markers.sort_values(by="auroc", inplace=True, ascending=False)


In [9]:
meta_markers

,group,cell_type,rank,gene,recurrence,auroc,fold_change,fold_change_detection,expression,precision,recall,population_size,n_datasets,zeng,macosko,Zeng_cluster,Macosko_cluster
49731,all,2664__Ex_C1ql4_Drd1_7,1,ENSMUSG00000071112,2,0.999987,495.800302,28.723223,1892.780792,0.000405,1.000000,69.5,2,True,True,2664,Ex_C1ql4_Drd1_7
29456,all,2134__Ex_Nkx2-1_Pmch_2,1,ENSMUSG00000035383,2,0.999920,6359.794762,144.799339,30735.364782,0.006814,1.000000,161.0,2,True,True,2134,Ex_Nkx2-1_Pmch_2
54014,all,2836__Ser_Fev_Siglec15,1,ENSMUSG00000020838,2,0.999841,1170.454954,295.510178,2415.064287,0.008180,1.000000,79.5,2,True,True,2836,Ser_Fev_Siglec15
54015,all,2836__Ser_Fev_Siglec15,2,ENSMUSG00000006764,2,0.999819,1079.995278,111.144299,3528.150528,0.002631,1.000000,79.5,2,True,True,2836,Ser_Fev_Siglec15
3153,all,1225__Inh_Lhx6_Esr2_12,1,ENSMUSG00000021055,2,0.999803,327.974485,74.468954,731.917665,0.001805,1.000000,95.5,2,True,True,1225,Inh_Lhx6_Esr2_12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159424,all,5283__Micro_Selplg_Siglech,988,ENSMUSG00000009214,2,0.506466,4.224739,6.853218,3.508549,0.285256,0.013881,108026.0,2,True,True,5283,Micro_Selplg_Siglech
150651,all,5243__Endo_Flt1_Kcnj8,677,ENSMUSG00000029762,2,0.506354,4.779207,4.365317,5.582736,0.028507,0.015429,21837.0,2,True,True,5243,Endo_Flt1_Kcnj8
159425,all,5283__Micro_Selplg_Siglech,989,ENSMUSG00000002076,2,0.506207,4.333343,2.902071,5.315763,0.079077,0.016980,108026.0,2,True,True,5283,Micro_Selplg_Siglech
150652,all,5243__Endo_Flt1_Kcnj8,678,ENSMUSG00000039956,2,0.506130,5.132251,3.966347,6.493908,0.024850,0.015393,21837.0,2,True,True,5243,Endo_Flt1_Kcnj8


In [10]:
second_best_hits = pd.read_csv(os.path.join(full_run_location, "aurocs_1v1.csv.gz"))

In [11]:
second_best_hits = second_best_hits.rename(columns={'Unnamed: 0': 'target'})

In [12]:
second_best_hits_melted = pd.melt(second_best_hits, id_vars=['target'], value_name="auroc", var_name="reference")

In [13]:
second_best_hits_melted = second_best_hits_melted.dropna()

In [14]:
second_best_hits_melted['reference_study'] = second_best_hits_melted['reference'].apply(lambda x: getStudyId(str(x)))
second_best_hits_melted['target_study'] = second_best_hits_melted['target'].apply(lambda x: getStudyId(str(x)))

In [15]:
# Filter rows where reference_study is not equal to target_study
second_best_hits_melted = second_best_hits_melted[second_best_hits_melted['reference_study'] != second_best_hits_melted['target_study']]

In [16]:
second_best_hits_melted['target'] = second_best_hits_melted['target'].str.rstrip()
second_best_hits_melted['reference'] = second_best_hits_melted['reference'].str.rstrip()

In [17]:
second_best_hits = second_best_hits_melted

In [18]:
base_data_folder

PosixPath('/vault/lfrench/mouse_brain_cluster_replicability/data')

In [ ]:
#load one small part into memory for testing
#adata_Macosko = sc.read_h5ad(os.path.join(base_data_folder, 'whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.cpm.part_11.h5ad'))



In [ ]:
#full dataset
adata_Macosko = sc.read_h5ad(os.path.join(base_data_folder, "whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad"))
sc.pp.normalize_total(adata_Macosko, target_sum=1e6, inplace=True)


In [ ]:
#convert to pandas dataframe to handle memory leak

In [ ]:
sparse_X = sparse.csr_matrix(adata_Macosko.X)
df = pd.DataFrame.sparse.from_spmatrix(sparse_X)

In [ ]:
df.index = adata_Macosko.obs.index  # Assuming row labels are stored in `.obs_names`
df.columns = adata_Macosko.var.index

In [ ]:
adata_Macosko_obs = adata_Macosko.obs
adata_Macosko = df

In [ ]:
adata_Macosko_obs['Annotation'] = adata_Macosko_obs['ClusterNm']

In [ ]:
Macosko_marker_gene_universe = set(adata_Macosko.columns)

In [ ]:
#filter for genes in the adata file - only needed when testing on a part file with limited genes
meta_markers = meta_markers[meta_markers.gene.isin(Macosko_marker_gene_universe)]

In [ ]:
def run_single_cluster(target_cluster, marker_table, best_vs_next = False):
    print(target_cluster)
    #result_rows = pd.DataFrame()
    #if targetting the second best then pull out the Zeng cluster to get the second best from a processed R output
    Macosko_target_cluster = "Macosko|" + target_cluster.split("__")[1]
    if best_vs_next:
        Zeng_target_cluster = "Zeng|" + target_cluster.split("__")[0]
        best_two = set(second_best_hits[second_best_hits['reference'] == Zeng_target_cluster]['target'])
        best_two.discard(Macosko_target_cluster) #this item may not be in the best two for this Zeng reference cluster!
        best_two = list(best_two)
        best_two.sort() #sorted to prevent random choices if there is multiple choices for second best
        Macosko_second_cluster = best_two.pop() 
        Macosko_second_cluster = Macosko_second_cluster.replace("Macosko|", "")        
    Macosko_target_cluster = Macosko_target_cluster.replace("Macosko|", "")
    #filter markers
    target_markers = marker_table[marker_table['cell_type'] == target_cluster].copy()
    
    if target_markers.empty:
        print("No markers")
        return pd.DataFrame() 
    
    #sort again to be sure
    target_markers.sort_values(by="auroc", inplace=True, ascending=False)
    #init a running sum of markers and set to first marker
    best_marker = target_markers.iloc[0, target_markers.columns.get_loc('gene')]
    labels = list(adata_Macosko_obs.Annotation == Macosko_target_cluster)

    genes_used = 1
    if best_vs_next:
        cell_universe = adata_Macosko_obs.Annotation.isin((Macosko_second_cluster, Macosko_target_cluster)).tolist()
        labels = [x for x, m in zip(labels, cell_universe) if m]
        running_sums = np.asarray(adata_Macosko[best_marker].loc[cell_universe])
        auc = get_auroc(running_sums, labels)
    else:
        running_sums = np.asarray(adata_Macosko[best_marker])
        auc = get_auroc(running_sums, labels)
        
    result_rows = pd.DataFrame([{'target_cluster': target_cluster, 
                                 'auroc': auc, 
                                 'current_marker': best_marker,
                                 'genes_used':genes_used, 'max_markers':target_markers.shape[0]}])
    
    ##for loop over markers minus the test marker
    ##is it the concat that is slow? -write each cell type out as a file? in a different thread?
    for target_marker in set(target_markers['gene'])-{best_marker}:
        genes_used+=1
        #print(genes_used) if genes_used % 100 == 0 else None
        ###add next marker
        #this is the slow part

        if best_vs_next:
            running_sums = running_sums + np.asarray(adata_Macosko[target_marker].loc[cell_universe])
            auc = get_auroc(running_sums, labels)
        else:
            running_sums = running_sums + np.asarray(adata_Macosko[target_marker])
            auc = get_auroc(running_sums, labels)

        ###get AUROC and store size and type
        new_row = {'target_cluster': target_cluster, 
                                 'auroc': auc, 
                                 'current_marker': target_marker,
                                 'genes_used':genes_used, 'max_markers':target_markers.shape[0]}
        #result_rows = pd.concat([result_rows, auroc])
        result_rows.loc[len(result_rows)] = new_row
    result_rows["best_vs_next"] = best_vs_next
    del running_sums
    return result_rows


In [ ]:
#%%time #test one cluster
run_single_cluster("5283__Micro_Selplg_Siglech", marker_table = meta_markers, best_vs_next=True)

In [ ]:
#%%time
#run_single_cluster("Zeng|1225__Macosko|Inh_Lhx6_Esr2_12", marker_table = meta_markers)

In [ ]:
#%%time
#dactyl has 96 procs - multithreading may be broken, restart notebook server? 
use_best_vs_next=True
markers_to_use = meta_markers #or Zeng_markers or Macosko_markers
#markers_to_use = Macosko_markers
#markers_to_use = Zeng_markers
print("Starting pool")
print("Using best vs next? " + str(use_best_vs_next))
pool = concurrent.futures.ThreadPoolExecutor(max_workers=3) #threading only helps AUC computation, which is small for best vs next
futures = []

for target_cluster in set(meta_markers.cell_type):
#for target_cluster in set(meta_markers.cell_type):
    future = pool.submit(run_single_cluster, target_cluster, markers_to_use, use_best_vs_next)
    futures.append(future)

pool.shutdown(wait=True)
print("Done running pool")
# Retrieve the results from the futures
results = []
for future in futures:
    result = future.result()
    results.append(result)

all_result_rows = pd.concat(results)

In [ ]:
print(all_result_rows.auroc.mean())

In [ ]:
all_result_rows.to_csv(os.path.join(here(), "results/MetaMarkers_" + metamarkers_location + "/MetaMarkers_greedy_auc_tests.Macosko." +str(round(time.time()))+ ".csv.gz"), index=False)


In [ ]:
print("Done Macosko!")

In [ ]:
print(os.path.join(here(), "results/MetaMarkers_" + metamarkers_location + "/MetaMarkers_greedy_auc_tests.Macosko." +str(round(time.time()))+ ".csv.gz"))

In [ ]:
###############################
###############################
###############################
###############################
#Zeng code below - duplication!! Needs refactoring
###############################
###############################
###############################
###############################

In [19]:
#part file for testing - skip the cluster info join below if used
#adata_Zeng = sc.read_h5ad(os.path.join(base_data_folder, 'whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.cpm.part_34.h5ad'))

In [ ]:
#full file
adata_Zeng = sc.read_h5ad(os.path.join(base_data_folder, 'whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad'))
adata_Zeng.var = adata_Zeng.var.set_index("gene_identifier")
sc.pp.normalize_total(adata_Zeng, target_sum=1e6, inplace=True)

In [20]:
adata_Zeng

AnnData object with n_obs × n_vars = 4042976 × 541
    obs: 'library_prep', 'gene.counts.0', 'doublet_score', 'roi', 'umi.counts', 'method', 'sex', 'external_donor_name', 'age', 'medical_conditions', 'cl', 'cluster_id', 'cluster_id_label', 'supertype_id', 'supertype_label', 'supertype_id_label', 'subclass_id', 'subclass_label', 'subclass_id_label', 'class_label', 'class_id', 'class_id_label', 'anatomical_annotation', 'notes', 'CCF_broad.freq', 'CCF_acronym.freq', 'v3.size', 'v2.size', 'multiome.size', 'cluster.markers', 'merfish.markers', 'F', 'M', 'sex.bias', 'Dark', 'Light', 'nt_type_label', 'nt_type_combo_label', 'study_id', 'filter_out'
    var: 'gene_symbol', 'in_Macosko'

In [ ]:
Zeng_cluster_info_AWS = pd.read_csv(os.path.join(base_data_folder, "whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv"), sep = '\t')
#set study ID here to prevent view as actual memory grabs 
Zeng_cluster_info_AWS['study_id'] = "Zeng"
#set it to be strings
adata_Zeng.obs['cl'] = adata_Zeng.obs['cl'].astype(str)
Zeng_cluster_info_AWS['cl'] = Zeng_cluster_info_AWS['cl'].astype(str)
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs = adata_Zeng.obs.merge(Zeng_cluster_info_AWS, how="left", on='cl')



In [22]:
#LQ or low quality cells
sum(adata_Zeng.obs['subclass_label'] == "LQ")
#remove #LQ
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs["filter_out"] = adata_Zeng.obs['subclass_label'] == "LQ"
adata_Zeng = adata_Zeng[~adata_Zeng.obs["filter_out"],:]
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)


/tmp/ipykernel_3165605/4001983129.py:7: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)


In [23]:
#no missing values in Zeng cell type annotations
adata_Zeng.obs['cl'].isna().sum()

0

In [24]:
adata_Zeng.shape

(4042976, 541)

In [25]:
sparse_X = sparse.csr_matrix(adata_Zeng.X)
df = pd.DataFrame.sparse.from_spmatrix(sparse_X)

In [26]:
df.index = adata_Zeng.obs.index  # Assuming row labels are stored in `.obs_names`
df.columns = adata_Zeng.var.index

In [27]:
adata_Zeng_obs = adata_Zeng.obs
adata_Zeng = df

In [28]:
#perform it for real
adata_Zeng = adata_Zeng[~adata_Zeng_obs["filter_out"]]
#also filter the observations
adata_Zeng_obs = adata_Zeng_obs[~adata_Zeng_obs["filter_out"]]

In [29]:
adata_Zeng_obs


,library_prep,gene.counts.0,doublet_score,roi,umi.counts,method,sex,external_donor_name,age,medical_conditions,...,merfish.markers,F,M,sex.bias,Dark,Light,nt_type_label,nt_type_combo_label,study_id,filter_out
0,L8TX_180406_01_F01,2951,0.047619,PAR-POST-PRE-SUB-ProS,5963.0,10Xv2,M,381293,53 days,Light,...,"Rxfp1,Glis3,Galr1,Bmp3",0.40,0.60,NaN,0.09,0.91,Glut,Glut,Zeng,False
1,L8TX_180406_01_C02,2938,0.030303,PAR-POST-PRE-SUB-ProS,6334.0,10Xv2,F,381295,54 days,Light,...,"Rxfp1,Glis3,Galr1,Bmp3",0.40,0.60,NaN,0.09,0.91,Glut,Glut,Zeng,False
2,L8TX_180406_01_B02,4271,0.298851,PAR-POST-PRE-SUB-ProS,11624.0,10Xv2,M,381293,53 days,Light,...,"Rxfp1,Glis3,Galr1,Bmp3",0.40,0.60,NaN,0.09,0.91,Glut,Glut,Zeng,False
3,L8TX_180406_01_F01,2974,0.011905,PAR-POST-PRE-SUB-ProS,6443.0,10Xv2,M,381293,53 days,Light,...,"Rxfp1,Glis3,Galr1,Bmp3",0.40,0.60,NaN,0.09,0.91,Glut,Glut,Zeng,False
4,L8TX_180406_01_C02,3207,0.030303,PAR-POST-PRE-SUB-ProS,7003.0,10Xv2,F,381295,54 days,Light,...,"Rxfp1,Glis3,Galr1,Bmp3",0.40,0.60,NaN,0.09,0.91,Glut,Glut,Zeng,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044531,L8XR_220728_01_A05,5509,0.000000,Mouse Multiome MB - VTA-SN,16885.0,10xMulti,M,641405,59 days,Light,...,"Slc17a8,Slc6a5,Arhgap28",0.36,0.64,NaN,0.09,0.91,GABA-Glyc,GABA-Glyc,Zeng,False
4044532,L8XR_220728_01_A05,8125,0.188406,Mouse Multiome MB - VTA-SN,55134.0,10xMulti,M,641405,59 days,Light,...,NaN,0.33,0.67,NaN,0.00,1.00,Glut,Glut,Zeng,False
4044533,L8XR_220728_01_A05,6459,0.028571,Mouse Multiome MB - VTA-SN,23478.0,10xMulti,M,641405,59 days,Light,...,"Slc6a5,Gata3,Nr2f1,Tox3,Syndig1,Tafa1,Kcnk9,Kc...",0.50,0.50,NaN,0.01,0.99,GABA-Glyc,GABA-Glyc,Zeng,False
4044534,L8XR_220630_02_B10,6352,0.243243,Mouse Multiome OLF - AON-TT-DP,28199.0,10xMulti,F,625156,59 days,Light,...,"Slc6a5,Gata3,Nr2f1,Tox3,Syndig1,Tafa1,Kcnk9,Kc...",0.50,0.50,NaN,0.01,0.99,GABA-Glyc,GABA-Glyc,Zeng,False


In [30]:
adata_Zeng_obs['Annotation'] = adata_Zeng_obs['cl'].astype(str)

In [31]:
Zeng_marker_gene_universe = set(adata_Zeng.columns)

In [32]:
#filter for genes in the adata file - only used when testing
meta_markers = meta_markers[meta_markers.gene.isin(Zeng_marker_gene_universe)]

In [34]:
len(Zeng_marker_gene_universe)

541

In [35]:
adata_Zeng.shape

(4042976, 541)

In [36]:
def run_single_cluster_Zeng(target_cluster, marker_table, best_vs_next = False):
    print(target_cluster)
    #result_rows = pd.DataFrame()
    #if targetting the second best then pull out the Zeng cluster to get the second best from a processed R output
    Macosko_target_cluster = "Macosko|" + target_cluster.split("__")[1]
    Zeng_target_cluster =  "Zeng|" + target_cluster.split("__")[0]
    if best_vs_next:
        best_two = set(second_best_hits[second_best_hits['reference'] == Macosko_target_cluster]['target'])
        best_two.discard(Zeng_target_cluster) #this item may not be in the best two for this Zeng reference cluster!
        best_two = list(best_two)
        best_two.sort() #sorted to prevent random choices if there is multiple choices for second best
        Zeng_second_cluster = best_two.pop() 
        Zeng_second_cluster = Zeng_second_cluster.replace("Zeng|", "")        
    Zeng_target_cluster = Zeng_target_cluster.replace("Zeng|", "")
    #filter markers
    target_markers = marker_table[marker_table['cell_type'] == target_cluster].copy()
    
    if target_markers.empty:
        print("No markers")
        return pd.DataFrame() 
    
    #sort again
    target_markers.sort_values(by="auroc", inplace=True, ascending=False)
    #init a running sum of markers and set to first marker
    best_marker = target_markers.iloc[0, target_markers.columns.get_loc('gene')]
    labels = list(adata_Zeng_obs.Annotation == Zeng_target_cluster)

    genes_used = 1
    if best_vs_next:
        cell_universe = adata_Zeng_obs.Annotation.isin((Zeng_second_cluster, Zeng_target_cluster)).tolist()
        labels = [x for x, m in zip(labels, cell_universe) if m]
        running_sums = np.asarray(adata_Zeng[best_marker].loc[cell_universe])
        auc = get_auroc(running_sums, labels)
    else:
        running_sums = np.asarray(adata_Zeng[best_marker])
        auc = get_auroc(running_sums, labels)
        
    result_rows = pd.DataFrame([{'target_cluster': target_cluster, 
                                 'auroc': auc, 
                                 'current_marker': best_marker,
                                 'genes_used':genes_used, 'max_markers':target_markers.shape[0]}])
    
    ##for loop over markers minus the test marker
    ##is it the concat that is slow? -write each cell type out as a file? in a different thread?
    for target_marker in set(target_markers['gene'])-{best_marker}:
        genes_used+=1
        #print(genes_used) if genes_used % 100 == 0 else None
        ###add next marker
        #this is the slow part

        if best_vs_next:
            running_sums = running_sums + np.asarray(adata_Zeng[target_marker].loc[cell_universe])
            auc = get_auroc(running_sums, labels)
        else:
            running_sums = running_sums + np.asarray(adata_Zeng[target_marker])
            auc = get_auroc(running_sums, labels)

        ###get AUROC and store size and type
        new_row = {'target_cluster': target_cluster, 
                                 'auroc': auc, 
                                 'current_marker': target_marker,
                                 'genes_used':genes_used, 'max_markers':target_markers.shape[0]}
        #result_rows = pd.concat([result_rows, auroc])
        result_rows.loc[len(result_rows)] = new_row
    result_rows["best_vs_next"] = best_vs_next
    del running_sums
    return result_rows


In [ ]:
#%%time
#run_single_cluster_Zeng("Microglia NN__MC_68 Microglia", marker_table = meta_markers, best_vs_next=True)

In [37]:
run_single_cluster_Zeng("5283__Micro_Selplg_Siglech", marker_table = meta_markers, best_vs_next=True)

5283__Micro_Selplg_Siglech


,target_cluster,auroc,current_marker,genes_used,max_markers,best_vs_next
0,5283__Micro_Selplg_Siglech,0.745465,ENSMUSG00000024013,1,31,True
1,5283__Micro_Selplg_Siglech,0.798892,ENSMUSG00000052142,2,31,True
2,5283__Micro_Selplg_Siglech,0.787915,ENSMUSG00000037321,3,31,True
3,5283__Micro_Selplg_Siglech,0.640010,ENSMUSG00000061232,4,31,True
4,5283__Micro_Selplg_Siglech,0.627287,ENSMUSG00000024397,5,31,True
5,5283__Micro_Selplg_Siglech,0.628620,ENSMUSG00000037548,6,31,True
6,5283__Micro_Selplg_Siglech,0.629837,ENSMUSG00000035929,7,31,True
7,5283__Micro_Selplg_Siglech,0.638574,ENSMUSG00000041538,8,31,True
8,5283__Micro_Selplg_Siglech,0.639280,ENSMUSG00000053835,9,31,True
9,5283__Micro_Selplg_Siglech,0.644401,ENSMUSG00000024334,10,31,True


In [ ]:
#%%time
#run_single_cluster_Zeng("Zeng|5249__Macosko|Endo_Flt1_Acta2_Olfr558_3", best_vs_next=False, marker_table = Macosko_markers)

In [ ]:
#%%time
#dactyl has 96 procs - multithreading may be broken, restart notebook server? 
use_best_vs_next=True
markers_to_use = meta_markers #or Zeng_markers
#markers_to_use = Macosko_markers #or Zeng_markers

print("Starting pool")
print("Using best vs next? " + str(use_best_vs_next))
pool = concurrent.futures.ThreadPoolExecutor(max_workers=4) #threading only helps AUC computation, which is small for best vs next
futures = []

#for target_cluster in set(meta_markers.head(20).cell_type):
for target_cluster in set(meta_markers.cell_type):
    future = pool.submit(run_single_cluster_Zeng, target_cluster, markers_to_use, use_best_vs_next)
    futures.append(future)

pool.shutdown(wait=True)
print("Done running pool")
# Retrieve the results from the futures
results = []
for future in futures:
    result = future.result()
    results.append(result)

all_result_rows = pd.concat(results)

In [ ]:
all_result_rows

In [ ]:
all_result_rows.auroc.mean()

In [ ]:
all_result_rows.to_csv(os.path.join(here(), "results/MetaMarkers_" + metamarkers_location + "/MetaMarkers_greedy_auc_tests.Zeng." +str(round(time.time()))+ ".csv.gz"), index=False)


In [ ]:
print("Done Zeng!")

In [38]:
os.path.join(here(), "results/MetaMarkers_" + metamarkers_location)

'/vault/lfrench/mouse_brain_cluster_replicability/results/MetaMarkers_2009_reciprocals'